# Quickstart

Get started here. We will assess a payment default prediction model for gender fairness using Lens, in 5 minutes. More in-depth information can be found in the [lens FAQ](https://credoai-lens.readthedocs.io/en/latest/notebooks/lens_faq.html#How-can-I-choose-which-assessments-to-run?)

## Setup

Setup instruction can be found on [readthedocs](https://credoai-lens.readthedocs.io/en/stable/setup.html)

**Find the code**

This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/quickstart.ipynb).

**Data + Model Preparation (before Lens)**

Some quick setup. This script reflects all of your datascience work before assessment and integration with Credo AI.

Here we have a gradient boosted classifier trained on the UCI Credit Card Default Dataset.

In [24]:
# model and df are defined by this script
%run training_script.py

### Imports

In [25]:
# Base credo imports
import credoai 
print(credoai.__version__)

# Import Lens and necessary artifacts
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData


1.0.0


## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model.
We will be evaluating model fairness according to three different metrics.

We'll break down what the artifacts are [below](#Breaking-Down-The-Steps).

In [31]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

In lens, the modules that evaluate models and/or datasets are called `evaluators`. Since in this case we are interested in evaluate model fairness, we will be importing the `ModelFairness` evaluator.

In [34]:
from credoai.evaluators import ModelFairness

In [42]:
# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

The next step is to add the evaluator to the initialized lens object. Notice that when we add an evaluator to the lens pipeline, it is initialized with all the necessary parameters.

In [43]:
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics), id='MyModelFairness')

2022-09-26 11:58:48,071 - lens - INFO - Evaluator Fairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX


Let's run the pipeline and visualize the results!


In [45]:
lens.run()
lens.get_results()['MyModelFairness'][0]

2022-09-26 11:59:14,096 - lens - INFO - Running evaluation for step: MyModelFairness


,type,value,subtype
0,equal_opportunity,0.015972,fairness
1,precision_score,0.012715,parity
2,recall_score,0.015972,parity


`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

## Adding multiple evaluators

If we want to add multiple evaluators to our pipeline, one way of doing it could be repeating the `add` step shown above. Another way is to define the pipeline steps, and pass it to `Lens` at initialization time. Let's explore the latter!

In [47]:
# Another evaluator that assess fairness in your dataset
from credoai.evaluators import DataFairness

In [48]:
pipeline = [
    (ModelFairness(metrics), "MyModelFairness"),
    (DataFairness(), "MyDataFairness"),
]
lens = Lens(model=credo_model, assessment_data=credo_data, pipeline=pipeline)

2022-09-26 12:15:50,006 - lens - INFO - Evaluator Fairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-09-26 12:15:50,010 - lens - INFO - Evaluator DataFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX


Above, each of the `tuples` in the `list` is in the form `(instantiated_evaluator, id)`.

In [51]:
lens.run()
results = lens.get_results()

2022-09-26 12:16:23,229 - lens - INFO - Running evaluation for step: MyModelFairness
2022-09-26 12:16:23,239 - lens - INFO - Running evaluation for step: MyDataFairness


Let's check that we have results for both of our evaluators.

In [56]:
results['MyModelFairness'][0]

,type,value,subtype
0,equal_opportunity,0.015972,fairness
1,precision_score,0.012715,parity
2,recall_score,0.015972,parity


In [57]:
results['MyDataFairness'][0]

,value,target,type,subtype
0,0.183853,NaN,sensitive_feature,prediction_score
1,0.027886,1.0,demographic_parity,difference
2,0.880573,1.0,demographic_parity,ratio
3,0.025036,NaN,proxy_mutual_information,max


## In-Depth Overview

CredoAI Lens is the assessment framework component of the broader CredoAI suite.
It is usable as a standalone gateway to assessments or in combination
with CredoAI's Governance Platform. 

Understanding how your AI systems are operating is the most important step in intervening upon your system. From the technically complicated questions of improving a system to the business-relevant questions of whether to deploy a system - everything is fundamentally founded upon good observability. Lens strives to make assessment comprehensive, easy, and adaptable. The primary outputs from Lens are **assessment results** in the form of various metrics.

When used in combination with CredoAI's Governance Platform, Lens can also be used to provide `evidences` to the Governance object. See !insert link to notebook! for more details on any governance related operations.

### Evaluators

Evaluators are the core units that perform any type of assessment on models and/or datasets. They can be developed internally by CredoAI, as well leverage existing frameworks that are alreayd part of the broader open-source ecosystem. Cusom analytics can be also easily wrapped into an evaluator (see `developer guide ! insert link`).

AI system assessment starts with verifying standard performance metrics to an evolving set of assessments falling under the banner of *Responsible AI*. A non-exhaustive list includes

* Fairness
* Explainability
* Performance
* Robustness
* Equity
* Privacy
* Security

These different categories of assessment differ substantially based on whether one is 
evaluating datasets or models, what kind of model (e.g., tabular, NLP, computer vision), and the use-case. As the ecosystem develops, Lens will support assessing a broader range of AI systems. Currently, we are focused on Fairness.

### Governance

While Lens is a stand-alone assessment framework, its value is increased when combined with the CredoAI Governance Platform. The platform supports multi-stakeholder `Alignment` on how to assess your AI systems (e.g., what does good look like for this system?). 

It also supports translating a set of requirements, defined either within an organization or by existing legistlation, into a cohesive report that is accessible to a range of diverse stakeholders.

Check out the [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/latest/notebooks/governance_integration.html) for information.



## Breaking Down The Steps

### Preparing artifacts

Lens interacts with Artifacts which wrap model and data objects and standardize them for use by different evaluators.

Below we create a `ClassifiactionModel` artifact. This is a light wrapper for any kind of fitted classification model-like object. 

We also create a `TabularData` artifact which stores X, y and sensitive features.

Both of these objects are customizable. See `lens_customization.ipynb` for more information.


In [62]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

#### Model

Model type objects, like `ClassificationModel` used above, serve as adapters between arbitrary models and the evaluators in Lens. Some evaluators depend on Model instantiating certain methods. For example, `ClassificationModel` can accept any generic object having `predict` and `predict_proba` methods, including fitted sklear pipelines. For more details see the source code: !!Insertlink!!


The user can make use of existing model artifacts or create their own. In order to do that, see deatails in !!insertlink!!
<br>



#### CredoData

_Data_ type artifact, like `TabularData` serve as adapters between datasets and the evaluators in Lens.

When you pass data to a _Data_ artifact, the artifact performs various steps of validation, and formats them so that they can be used by evaluators. The aim of this procedure is to preempt errors down the line.

You can pass Data to Lens as a **training dataset** or an **assessment dataset** (see lens class documentation). If the former, it will not be used to assess the model. Instead, dataset assessments will be performed on the dataset (e.g., fairness assessment). The validation dataset will be assessed in the same way, but _also_ used to assess the model, if provided.

Similarly to _Model_ type objects, _Data_ objects can be customized, see !!insertlink!!

### Evaluators 

Lens uses the above artifacts to ensure a successfull run of the evaluators. As we have seen in the sections [Lens in 5 minutes](##Lens-in-5-minutes) and [Adding multiple evaluators](##Adding-multiple-evaluators), multiple evaluators can be added to _Lens_ pipeline. Each evaluators contains information on what it needs in order to run successfully, and it executes a validation step at _add_ time.

The result of the validation depends on what artifacts are available, their content and the type of evaluator being added to the pipeline. In case the validation process fails, the user is notified the reason why the evaluator cannot be added to the pipeline.

See for example:

In [58]:
from credoai.evaluators import Privacy
lens.add(Privacy())

2022-09-26 15:14:27,368 - lens - INFO - Evaluator Privacy NOT added to the pipeline: Missing object training_data


Currently no automatic run of evaluators is supported. However, when Lens is used in combination with Credo AI Platform, it is possible to download an assessment plan which then gets converted into a set of evaluations that Lens can run programmatically. For more information: !!insert link!!

### Run Lens

After we have initialized _Lens_ the _Model_ and _Data_ (`ClassificationModel` and `TabularData` in our example) type artifacts, we can add whichever evaluators we want to the pipeline, and finally run it!

In [63]:
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics), id='MyModelFairness')
lens.run()

2022-09-26 15:48:33,916 - lens - INFO - Evaluator Fairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-09-26 15:48:33,918 - lens - INFO - Running evaluation for step: MyModelFairness


As you can notice, when adding _evaluators_ to lens, they need to be instantiated. If any extra arguments need to be passed to the evaluator (like metrics in this case), this is the time to do it.

**Getting Assessment Results**

Afte the pipeline is run, the results become accessible via the method `get_results()`

`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

In [64]:
lens.get_results()

{'MyModelFairness': [                type     value   subtype
  0  equal_opportunity  0.013653  fairness
  1    precision_score  0.019533    parity
  2       recall_score  0.013653    parity]}

**Credo AI Governance Platform**

For information on how to interact with the plaform, please look into: [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/latest/notebooks/governance_integration.html) tutorial for directions.
